# Cloud Workshop Azure Databricks
## 07. Classification Evaluation

## Evaluating a Classification Model

In this exercise, you will create a pipeline for a classification model, and then apply commonly used metrics to evaluate the resulting classifier.

### Prepare the Data

First, import the libraries you will need and prepare the training and test data:

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Load the source data
# File location and type
file_location = "/FileStore/tables/flights.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(data)

# Split the data
splits = data.randomSplit([0.7, 0.3])

train = splits[0]
test = splits[1]

DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay,Late
21,2,WN,10721,13342,26,57,1
13,1,AA,15016,12892,51,27,1
5,5,FL,10397,11433,9,4,0
22,1,US,11278,14100,35,71,1
23,4,WN,12451,10693,9,5,0
5,7,AA,11298,15016,39,42,1
4,6,UA,13930,14307,71,58,1
10,3,9E,14307,11433,68,140,1
29,7,UA,14057,14771,130,125,1
14,7,UA,14771,11292,20,42,1


### Define the Pipeline and Train the Model
Now define a pipeline that creates a feature vector and trains a classification model

In [5]:
monthdayIndexer = StringIndexer(inputCol="DayofMonth", outputCol="DayofMonthIdx")
weekdayIndexer = StringIndexer(inputCol="DayOfWeek", outputCol="DayOfWeekIdx")
carrierIndexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIdx")
originIndexer = StringIndexer(inputCol="OriginAirportID", outputCol="OriginAirportIdx")
destIndexer = StringIndexer(inputCol="DestAirportID", outputCol="DestAirportIdx")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normNums")
featVect = VectorAssembler(inputCols=["DayofMonthIdx", "DayOfWeekIdx", "CarrierIdx", "OriginAirportIdx", "DestAirportIdx", "normNums"], outputCol="features")

lr = LogisticRegression(labelCol="Late", featuresCol="features")

pipeline = Pipeline(stages=[monthdayIndexer, weekdayIndexer, carrierIndexer, originIndexer, destIndexer, numVect, minMax, featVect, lr])

model = pipeline.fit(train)

### Test the Model
Now you're ready to apply the model to the test data.

In [7]:
prediction = model.transform(test)
predicted = prediction.select("features", col("prediction").cast("Int"), col("Late").alias("trueLabel"))

predicted.show(100, truncate=False)

+---------------------------------------------+----------+---------+
features |prediction|trueLabel|
+---------------------------------------------+----------+---------+
[25.0,2.0,10.0,1.0,25.0,0.7263681592039801] |1 |1 |
[25.0,2.0,10.0,57.0,33.0,0.3582089552238806] |1 |1 |
[25.0,2.0,10.0,11.0,40.0,0.23383084577114427]|0 |0 |
[25.0,2.0,10.0,37.0,5.0,0.5323383084577115] |1 |1 |
[25.0,2.0,10.0,14.0,11.0,0.263681592039801] |0 |1 |
[25.0,2.0,10.0,46.0,32.0,0.22885572139303484]|0 |0 |
[25.0,2.0,10.0,17.0,19.0,0.7064676616915423] |1 |1 |
[25.0,2.0,10.0,17.0,40.0,0.22388059701492538]|0 |0 |
[25.0,2.0,10.0,54.0,15.0,0.6368159203980099] |1 |1 |
[25.0,2.0,10.0,0.0,16.0,0.5522388059701493] |1 |1 |
[25.0,2.0,10.0,64.0,10.0,0.5024875621890548] |1 |1 |
[25.0,2.0,10.0,67.0,15.0,0.26865671641791045]|0 |0 |
[25.0,2.0,2.0,1.0,24.0,0.4427860696517413] |1 |1 |
[25.0,2.0,2.0,27.0,2.0,0.2537313432835821] |0 |1 |
[25.0,2.0,2.0,11.0,24.0,0.4626865671641791] |1 |1 |
[25.0,2.0,2.0,36.0,5.0,0.2537313432835821] |0 |0 |
[25.0,2.0,2.0,23.0,24.0,0.25870646766169153] |0 |1 |
[25.0,2.0,2.0,3.0,46.0,0.2885572139303483] |0 |0 |
[25.0,2.0,2.0,3.0,0.0,0.472636815920398] |1 |1 |
[25.0,2.0,2.0,3.0,2.0,0.6318407960199005] |1 |1 |
[25.0,2.0,2.0,3.0,13.0,0.24875621890547264] |0 |0 |
[25.0,2.0,2.0,3.0,14.0,0.8905472636815921] |1 |1 |
[25.0,2.0,2.0,3.0,58.0,0.2736318407960199] |0 |0 |
[25.0,2.0,2.0,3.0,1.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,3.0,1.0,0.39303482587064675] |1 |0 |
[25.0,2.0,2.0,3.0,33.0,0.6019900497512438] |1 |1 |
[25.0,2.0,2.0,3.0,61.0,0.2736318407960199] |0 |0 |
[25.0,2.0,2.0,3.0,41.0,0.2885572139303483] |0 |0 |
[25.0,2.0,2.0,3.0,62.0,0.2835820895522388] |0 |0 |
[25.0,2.0,2.0,47.0,1.0,0.582089552238806] |1 |1 |
[25.0,2.0,2.0,53.0,5.0,0.23383084577114427] |0 |0 |
[25.0,2.0,2.0,53.0,5.0,0.6567164179104478] |1 |1 |
[25.0,2.0,2.0,2.0,24.0,0.9154228855721394] |1 |1 |
[25.0,2.0,2.0,2.0,33.0,0.29850746268656714] |0 |0 |
[25.0,2.0,2.0,21.0,10.0,0.9402985074626866] |1 |1 |
[25.0,2.0,2.0,21.0,2.0,0.29850746268656714] |0 |1 |
[25.0,2.0,2.0,21.0,13.0,0.582089552238806] |1 |1 |
[25.0,2.0,2.0,21.0,14.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,21.0,6.0,0.2537313432835821] |0 |0 |
[25.0,2.0,2.0,21.0,28.0,0.5323383084577115] |1 |1 |
[25.0,2.0,2.0,35.0,24.0,0.24875621890547264] |0 |1 |
[25.0,2.0,2.0,54.0,5.0,0.21393034825870647] |0 |0 |
[25.0,2.0,2.0,0.0,4.0,0.7164179104477612] |1 |1 |
[25.0,2.0,2.0,0.0,24.0,0.23383084577114427] |0 |0 |
[25.0,2.0,2.0,0.0,29.0,0.5223880597014925] |1 |1 |
[25.0,2.0,2.0,0.0,28.0,0.6567164179104478] |1 |1 |
[25.0,2.0,2.0,6.0,24.0,0.38308457711442784] |1 |1 |
[25.0,2.0,2.0,32.0,1.0,0.572139303482587] |1 |1 |
[25.0,2.0,2.0,62.0,5.0,0.5522388059701493] |1 |1 |
[25.0,2.0,2.0,42.0,5.0,0.21890547263681592] |0 |0 |
[25.0,2.0,2.0,42.0,5.0,0.22885572139303484] |0 |0 |
[25.0,2.0,2.0,5.0,12.0,0.24378109452736318] |0 |1 |
[25.0,2.0,2.0,5.0,12.0,0.263681592039801] |0 |0 |
[25.0,2.0,2.0,5.0,1.0,0.24378109452736318] |0 |0 |
[25.0,2.0,2.0,5.0,1.0,0.3383084577114428] |1 |1 |
[25.0,2.0,2.0,29.0,24.0,0.7213930348258707] |1 |1 |
[25.0,2.0,12.0,69.0,17.0,0.25870646766169153]|0 |0 |
[25.0,2.0,12.0,69.0,17.0,0.26865671641791045]|0 |0 |
[25.0,2.0,12.0,7.0,34.0,0.21890547263681592] |0 |0 |
[25.0,2.0,12.0,2.0,34.0,0.2537313432835821] |0 |0 |
[25.0,2.0,12.0,22.0,17.0,0.417910447761194] |1 |1 |
[25.0,2.0,12.0,5.0,17.0,0.2537313432835821] |0 |0 |
[25.0,2.0,7.0,11.0,22.0,0.5572139303482587] |1 |1 |
[25.0,2.0,7.0,11.0,63.0,0.6069651741293532] |1 |1 |
[25.0,2.0,7.0,11.0,18.0,0.2935323383084577] |0 |1 |
[25.0,2.0,7.0,11.0,18.0,0.6865671641791045] |1 |1 |
[25.0,2.0,7.0,11.0,33.0,0.3681592039800995] |1 |0 |
[25.0,2.0,7.0,11.0,33.0,0.6069651741293532] |1 |1 |
[25.0,2.0,7.0,23.0,26.0,0.40298507462686567] |1 |1 |
[25.0,2.0,7.0,23.0,28.0,0.263681592039801] |0 |1 |
[25.0,2.0,7.0,10.0,26.0,0.2537313432835821] |0 |0 |
[25.0,2.0,7.0,10.0,14.0,0.7114427860696517] |1 |1 |
[25.0,2.0,7.0,26.0,7.0,0.24378109452736318] |0 |1 |
[25.0,2.0,7.0,26.0,7.0,0.746268656716418] |1 |1 |
[25.0

### Compute Confusion Matrix Metrics
Classifiers are typically evaluated by creating a *confusion matrix*, which indicates the number of:
- True Positives
- True Negatives
- False Positives
- False Negatives

From these core measures, other evaluation metrics such as *precision* and *recall* can be calculated.

In [9]:
tp = float(predicted.filter("prediction == 1.0 AND truelabel == 1").count())
fp = float(predicted.filter("prediction == 1.0 AND truelabel == 0").count())
tn = float(predicted.filter("prediction == 0.0 AND truelabel == 0").count())
fn = float(predicted.filter("prediction == 0.0 AND truelabel == 1").count())

metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", tp / (tp + fp)),
 ("Recall", tp / (tp + fn))],["metric", "value"])


In [10]:
display(metrics)

metric,value
TP,89291.0
FP,7226.0
TN,81651.0
FN,13143.0
Precision,0.9251323601023654
Recall,0.8716929925610637


### View the Raw Prediction and Probability
The prediction is based on a raw prediction score that describes a labelled point in a logistic function. This raw prediction is then converted to a predicted label of 0 or 1 based on a probability vector that indicates the confidence for each possible label value (in this case, 0 and 1). The value with the highest confidence is selected as the prediction.

In [12]:
prediction.select("rawPrediction", "probability", "prediction", col("Late").alias("trueLabel")).show(100, truncate=False)

+------------------------------------------+------------------------------------------+----------+---------+
rawPrediction |probability |prediction|trueLabel|
+------------------------------------------+------------------------------------------+----------+---------+
[-10.266709369435299,10.266709369435299] |[3.4770398866545655E-5,0.9999652296011334]|1.0 |1 |
[-0.44781862447420906,0.44781862447420906]|[0.38987953262334796,0.610120467376652] |1.0 |1 |
[2.542663784097164,-2.542663784097164] |[0.9270791125188385,0.07292088748116143] |0.0 |0 |
[-5.298788498514826,5.298788498514826] |[0.004972792643224975,0.9950272073567751] |1.0 |1 |
[1.5040022561030328,-1.5040022561030328] |[0.8181706399631788,0.1818293600368211] |0.0 |1 |
[2.7888585236791377,-2.7888585236791377] |[0.9420707818780343,0.057929218121965845] |0.0 |0 |
[-9.726800286846112,9.726800286846112] |[5.96593336563019E-5,0.9999403406663437] |1.0 |1 |
[2.8314312128466206,-2.8314312128466206] |[0.9443508634631637,0.05564913653683632] |0.0 |0 |
[-7.774206019002209,7.774206019002209] |[4.202644626797419E-4,0.9995797355373204] |1.0 |1 |
[-5.908028674049298,5.908028674049298] |[0.002710173762934345,0.9972898262370657] |1.0 |1 |
[-4.330841239488997,4.330841239488997] |[0.012985629969893187,0.987014370030107] |1.0 |1 |
[1.719055586859267,-1.719055586859267] |[0.8480071499697541,0.1519928500302459] |0.0 |0 |
[-2.5975734895432936,2.5975734895432936] |[0.06929474979346452,0.9307052502065355] |1.0 |1 |
[2.1659283904428897,-2.1659283904428897] |[0.8971478716727338,0.10285212832726613] |0.0 |1 |
[-3.049725898845532,3.049725898845532] |[0.04522930868755415,0.954770691312446] |1.0 |1 |
[2.2474325019960935,-2.2474325019960935] |[0.9044288377924219,0.09557116220757807] |0.0 |0 |
[2.2373949473441943,-2.2373949473441943] |[0.9035576894253289,0.09644231057467119] |0.0 |1 |
[1.5812437301810958,-1.5812437301810958] |[0.8293805883868104,0.17061941161318966] |0.0 |0 |
[-3.5915820666675504,3.5915820666675504] |[0.02681580112141122,0.9731841988785888] |1.0 |1 |
[-7.644596655121834,7.644596655121834] |[4.7839435079344034E-4,0.9995216056492066]|1.0 |1 |
[2.267208382400545,-2.267208382400545] |[0.9061245939843897,0.0938754060156103] |0.0 |0 |
[-14.142629093503936,14.142629093503936] |[7.209977542035055E-7,0.9999992790022458] |1.0 |1 |
[2.0839472209339824,-2.0839472209339824] |[0.8893331151187784,0.11066688488122159] |0.0 |0 |
[2.2736493228574766,-2.2736493228574766] |[0.9066710471907431,0.09332895280925693] |0.0 |0 |
[-1.5449339112161518,1.5449339112161518] |[0.17581917359739246,0.8241808264026075] |1.0 |0 |
[-6.568696659331177,6.568696659331177] |[0.0014016582194539516,0.998598341780546] |1.0 |1 |
[2.114158512770363,-2.114158512770363] |[0.8922717141058676,0.10772828589413239] |0.0 |0 |
[1.530891577120462,-1.530891577120462] |[0.8221367251374453,0.17786327486255474] |0.0 |0 |
[1.8696567277775813,-1.8696567277775813] |[0.8664185530141499,0.13358144698585] |0.0 |0 |
[-6.13104111131606,6.13104111131606] |[0.0021695986835513366,0.9978304013164487]|1.0 |1 |
[2.8534633704487993,-2.8534633704487993] |[0.9454974329380077,0.05450256706199224] |0.0 |0 |
[-7.965855792759816,7.965855792759816] |[3.4699407709510183E-4,0.9996530059229048]|1.0 |1 |
[-14.684054528585696,14.684054528585696] |[4.195618232881197E-7,0.9999995804381767] |1.0 |1 |
[1.1900567144277812,-1.1900567144277812] |[0.7667512074053945,0.23324879259460551] |0.0 |0 |
[-15.353186254543344,15.353186254543344] |[2.1487992151470657E-7,0.9999997851200786]|1.0 |1 |
[0.9861582070762527,-0.9861582070762527] |[0.7283284288113003,0.2716715711886996] |0.0 |1 |
[-6.158375200930248,6.158375200930248] |[0.0021112213627341864,0.9978887786372659]|1.0 |1 |
[2.507150560248772,-2.507150560248772] |[0.9246415829910782,0.07535841700892175] |0.0 |0 |
[2.1720148997468485,-2.1720148997468485] |[0.8977081395690802,0.10229186043091978] |0.0 |0 |
[-4.734457663723804,4.734457663723804] |[0.008710670644419147,0.9912893293555808] |1.0 |1 |
[2.5603575892382002,-2.5603575892382002] |[0

Note that the results include rows where the probability for 0 (the first value in the **probability** vector) is only slightly higher than the probability for 1 (the second value in the **probability** vector). The default *discrimination threshold* (the boundary that decides whether a probability is predicted as a 1 or a 0) is set to 0.5; so the prediction with the highest probability is always used, no matter how close to the threshold.

### Review the Area Under ROC
Another way to assess the performance of a classification model is to measure the area under a *received operator characteristic (ROC) curve* for the model. The **spark.ml** library includes a **BinaryClassificationEvaluator** class that you can use to compute this. A ROC curve plots the True Positive and False Positive rates for varying threshold values (the probability value over which a class label is predicted). The area under this curve gives an overall indication of the models accuracy as a value between 0 and 1. A value under 0.5 means that a binary classification model (which predicts one of two possible labels) is no better at predicting the right class than a random 50/50 guess.

In [15]:
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(prediction)

print ("AUC = ", auc)

AUC = 0.9483764281212743

### Review the AreaUnderPR

In [17]:
evaluator = BinaryClassificationEvaluator(labelCol="Late", rawPredictionCol="rawPrediction", metricName="areaUnderPR")
areaUnderPR = evaluator.evaluate(prediction)
print ("areaUnderPR = ", areaUnderPR)

areaUnderPR = 0.9645236563777777

> Fin